In [1]:
import numpy as np
import math

In [2]:
class PDF(object):
                
    def __init__(self):
        self.name = "BinnedTemplate"
        self.nps = {}
        self.samples = {}
        self.nominal = None
        self.IsDataSet = False
    def addSample(self, sample):
        self.samples[sample.name] = sample
        if self.nominal is None:
            self.nominal = sample.nominal
        else:
            self.nominal +=sample.nominal
        for nuis in sample.nps:
            self.nps[nuis] = sample.nps[nuis]
    def setData(self, data):
        self.IsDataSet = True
        self.data = np.asarray(data)
    def pdf(self, params):
        flux = np.zeros(len(self.nominal))
        for sample in self.samples:
            sample = self.samples[sample]
            for par in params:
                if par[0] in sample.nps:
                    flux += sample.evaluate(par[0],par[1])
        return self.nominal + flux
    
    def getIntegral(self, hist):
        return sum(hist)

In [3]:
class Sample(object):
    def __init__(self, name):
        self.name = name
        self.nps = {}
    def addHist(self,hist):
        assert isinstance(hist, list)
        self.nominal = np.asarray(hist)
        
    def setHistoSys(self, name, uphist, downhist):
        self.nps[name] = self.parametrize(len(self.nominal), uphist, downhist)

    def parametrize(self, length, up, down):
        func = np.zeros(length, dtype={'names':('nominal','up','down'),'formats':('f8','f8','f8')})
        func['up'] = up
        func['down'] = down
        func['nominal'] = self.nominal
        return func
    
    def PiecewiseLinear(self, alpha, I0, Iup, Idown):
        if alpha < 0:
            return (alpha*(I0-Idown))
        else:
            return (alpha*(Iup - I0))
        
    def evaluate(self, name, value):
        f = np.vectorize(self.PiecewiseLinear)
        func = self.nps[name]
        return f(value, func['nominal'],func['up'],func['down'])

## Test adding nominal to first sample

In [4]:
nominal = [10.,20.,30.,40.]
sig = Sample('signal')
sig.addHist(nominal)
print sig.nominal

[ 10.  20.  30.  40.]


## Add first systematic to sample

In [5]:
sig.setHistoSys('first',[i*1.5 for i in nominal],[i*.5 for i in nominal])
print sig.nps

{'first': array([( 10.,  15.,   5.), ( 20.,  30.,  10.), ( 30.,  45.,  15.),
       ( 40.,  60.,  20.)],
      dtype=[('nominal', '<f8'), ('up', '<f8'), ('down', '<f8')])}


In [6]:
print sig.nps['first']['up']
print sig.nominal
print sig.nps['first']['down']

[ 15.  30.  45.  60.]
[ 10.  20.  30.  40.]
[  5.  10.  15.  20.]


### test evaluation

In [7]:
flux = sig.evaluate("first",-.5)
print flux

[ -2.5  -5.   -7.5 -10. ]


In [8]:
print sig.nominal + flux

[  7.5  15.   22.5  30. ]


In [9]:
sig.setHistoSys('second',[(i*.1+1)*a for i,a in enumerate(nominal)],[(i*.1+1)*a for i,a in enumerate(nominal)])

In [10]:
print sig.nps['second']['up']
print sig.nominal
print sig.nps['second']['down']

[ 10.  22.  36.  52.]
[ 10.  20.  30.  40.]
[ 10.  22.  36.  52.]


In [11]:
sig.setHistoSys('third',[a+math.pow(a,4-i)*.001 for i,a in enumerate(nominal)],[a-math.pow(a,4-i)*.001 for i,a in enumerate(nominal)])

In [12]:
print sig.nps['third']['up']
print sig.nominal
print sig.nps['third']['down']

[ 20.    28.    30.9   40.04]
[ 10.  20.  30.  40.]
[  0.    12.    29.1   39.96]


In [13]:
params = [('first',-.5),('second',.3),('third',1.2)]
flux = np.zeros(len(sig.nominal))
for par in params:
    flux += sig.evaluate(par[0],par[1])
print sig.nominal + flux

[ 19.5    25.2    25.38   33.648]


In [14]:
bkgnominal = [40.,40.,40.,40.]
bkg = Sample('background')
bkg.addHist(bkgnominal)
print bkg.nominal

[ 40.  40.  40.  40.]


In [15]:
bkg.setHistoSys('first',[i*1.5 for i in nominal],[i*.5 for i in nominal])
print bkg.nps

{'first': array([( 40.,  15.,   5.), ( 40.,  30.,  10.), ( 40.,  45.,  15.),
       ( 40.,  60.,  20.)],
      dtype=[('nominal', '<f8'), ('up', '<f8'), ('down', '<f8')])}


In [16]:
pdf = PDF()
pdf.addSample(sig)
pdf.addSample(bkg)

In [17]:
params = [('first',-.5),('second',-.3),('third',1.2)]
pdf.pdf(params)

array([ 42.   ,  50.2  ,  52.88 ,  63.648])

# Testing minimization

In [18]:
from scipy.optimize import minimize

class minimizer(object):
    def __init__(self, name, opt = 'minuit'):
        self.name = name
        if opt.lower() in ['minuit','scipy']:
            self.type = opt.lower()
        else: self.type = 'minuit'
    def NLL(self, model, params):
        return (model.data*np.log(model.pdf(params))).sum()
    
    def newNLL(self, model):
        self.model = model
        names = model.nps.keys()
        init = np.ones(len(names))
        result = minimize(self.f, init, args=names, method = 'Nelder-Mead')
        if result.success:
            return zip(names,result.x)
        else:
            raise ValueError(result.message)

    def f(self, values, names):
        model = self.model
        params = zip(names, values)
        expected = model.pdf(params)
        return -(np.sum(model.data*np.log(expected)) - np.sum(expected))

In [19]:
pdf.setData([35.,40.,45.,50.])

In [20]:
opt = minimizer('test')
print opt.NLL(pdf, params)

673.688566641


In [21]:
print 'result is',opt.newNLL(pdf)

result is [('second', 1.7595483534248184e-07), ('third', 1.2248921934744734), ('first', -0.70429106865340052)]


# Test Channels!

In [22]:
class HistiModel:
    def __init__(self, name="ANewHistiModel"):
        self.name = name
        self.Channels = {}
        self.nps = {}
    def AddChannel(self, channel):
        if channel.IsDataSet:
            self.Channels[channel.name] = channel
        else:
            print "No Data Set - Using Asimov"
            #To Do impliment Asimov

        for norm in channel.nps:
            self.nps[norm] = channel.nps[norm]

In [31]:
amodel = HistiModel()
amodel.AddChannel(pdf)

Redefine for Models not Channels

In [30]:
class minimizer(object):
    def __init__(self, name, opt = 'minuit'):
        self.name = name
        if opt.lower() in ['minuit','scipy']:
            self.type = opt.lower()
        else: self.type = 'minuit'
#    def NLL(self, model, params):
#        return (model.data*np.log(model.pdf(params))).sum()
    
    def newNLL(self, model):
        self.model = model
        names = model.nps.keys()
        init = np.ones(len(names))
#        ranges = [[i*.1,i*10.] for i in init]
        result = minimize(self.f, init, args=names, method = 'BFGS')
        if result.success:
            return zip(names,result.x)
        else:
            raise ValueError(result.message)

    def f(self, values, names):
        model = self.model
        params = zip(names, values)
        nll = None
        for chan in model.Channels:
            thechannel = model.Channels[chan]
            expected = thechannel.pdf(params)
            if nll:
                nll -= (np.sum(thechannel.data*np.log(expected)) - np.sum(expected))
            else:
                nll = -(np.sum(thechannel.data*np.log(expected)) - np.sum(expected))
        return nll
        

In [32]:
nominal = [1.,1.,4.]
sig = Sample('newsignal')
sig.addHist(nominal)
print sig.nominal

sig.setHistoSys('first',[i*1.5 for i in nominal],[i*.5 for i in nominal])
print sig.nps

print sig.nps['first']['up']
print sig.nominal
print sig.nps['first']['down']

bkgnominal = [40.,40.,40.]
bkg = Sample('background')
bkg.addHist(bkgnominal)
print bkg.nominal
bkg.setHistoSys('first',[i*1.5 for i in bkgnominal],[i*.5 for i in bkgnominal])
print bkg.nps
bkg.setHistoSys('second',[(i*.1+1)*a for i,a in enumerate(nominal)],[(i*.1+1)*a for i,a in enumerate(nominal)])
bkg.setHistoSys('third',[a+math.pow(a,4-i)*.001 for i,a in enumerate(nominal)],[a-math.pow(a,4-i)*.001 for i,a in enumerate(nominal)])

[ 1.  1.  4.]
{'first': array([( 1.,  1.5,  0.5), ( 1.,  1.5,  0.5), ( 4.,  6. ,  2. )],
      dtype=[('nominal', '<f8'), ('up', '<f8'), ('down', '<f8')])}
[ 1.5  1.5  6. ]
[ 1.  1.  4.]
[ 0.5  0.5  2. ]
[ 40.  40.  40.]
{'first': array([( 40.,  60.,  20.), ( 40.,  60.,  20.), ( 40.,  60.,  20.)],
      dtype=[('nominal', '<f8'), ('up', '<f8'), ('down', '<f8')])}


In [33]:
ctlpdf = PDF()
ctlpdf.addSample(sig)
ctlpdf.addSample(bkg)
ctlpdf.setData([49.,50.,45.])

In [34]:
amodel.AddChannel(ctlpdf)

In [35]:
opt = minimizer('test')
#print opt.NLL(amodel, params)

In [36]:
print 'result is',opt.newNLL(amodel)

result is

/Users/vince/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in log


ValueError: Desired error not necessarily achieved due to precision loss.